# 🏋️‍♂️ Gym Dashboard Tutorial

This notebook will help you understand and test each component of your Gym Dashboard. We'll go through:

1. Setup and Configuration
2. Data Loading and Processing
3. Garmin Integration
4. Data Analysis Functions
5. Visualization Components

Let's get started!

## 1. Setup and Configuration

First, let's import all necessary libraries and set up our environment.

In [9]:
import sys
import os
sys.path.append('../src')

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
from dotenv import load_dotenv
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from garmin_integration import GarminDataLoader

# Load environment variables
load_dotenv('../.env')

print("Environment loaded successfully!")

Environment loaded successfully!


## 2. Data Loading and Processing

### 2.1 Google Sheets Integration

In [10]:
def load_sheet_data():
    """Load and process Google Sheets data"""
    try:
        scope = ['https://spreadsheets.google.com/feeds',
                'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name(
            '../credentials/credentials.json', scope)
        client = gspread.authorize(creds)
        
        sheet = client.open_by_key(os.getenv('SHEET_ID')).sheet1
        data = sheet.get_all_records()
        return pd.DataFrame(data)
    except Exception as e:
        print(f"Error loading Google Sheets data: {str(e)}")
        return pd.DataFrame()

# Test loading data
sheets_data = load_sheet_data()
print(f"Loaded {len(sheets_data)} rows from Google Sheets")
sheets_data.head()

Loaded 57 rows from Google Sheets


,Date,Exercise\t,Reps,Weight,Time seconds
0,22/02/2025,Negative pull-up,7;7;6;7,Body;Body;Body;BodyBand,
1,22/02/2025,Standing Cable Fly,8; 8; 8; 8; 8,17.5; 27.5; 27.5; 17.5; 12.5,
2,22/02/2025,Seated Cable Row,10; 10; 10; 9,100; 100;100;100,
3,22/02/2025,Lat Pulldowns,10;10;10;10,100; 85;85;85,
4,22/02/2025,Incline Abs,15;15;15,Body,


### 2.2 Data Processing Functions

Let's test our data processing functions that handle special cases like body weight exercises.

In [19]:
def process_workout_data(df):
    """Process workout data with special cases"""
    # Clean column names
    df.columns = [col.strip().replace('\t', '') for col in df.columns]
    
    # Convert date format
    df['date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
    
    def parse_reps(x):
        if pd.isna(x) or str(x).strip() == '':
            return []
        return [int(r.strip()) for r in str(x).split(';') if r.strip().isdigit()]
    
    def parse_weights(x, num_sets):
        if pd.isna(x) or str(x).strip() == '':
            return []
        
        weights = []
        weight_items = str(x).split(';')
        
        # If there's just a single 'Body' value and multiple sets
        if len(weight_items) == 1 and 'body' in weight_items[0].lower() and num_sets > 1:
            base_weight = 70  # Default body weight
            if 'band' in weight_items[0].lower():
                return [base_weight + 10] * num_sets  # Body weight + band resistance for each set
            else:
                return [base_weight] * num_sets  # Just body weight for each set
        
        # Normal processing for multiple weights or non-body weight cases
        for w in weight_items:
            w = w.strip().lower()
            if 'body' in w:
                base_weight = 120  # Default body weight is 2/3 my body weight in LBS
                if 'band' in w:
                    weights.append(base_weight + 10)  # Body weight + band resistance
                else:
                    weights.append(base_weight)
            elif w.replace('.', '').isdigit():
                weights.append(float(w))
        
        return weights
    
    # Count sets (number of comma-separated values in Reps)
    df['sets'] = df['Reps'].apply(lambda x: len(str(x).split(';')) if not pd.isna(x) and str(x).strip() != '' else 0)
    
    # Process reps
    df['reps_list'] = df['Reps'].apply(parse_reps)
    
    # Process weights with set count information
    df['weights_list'] = df.apply(lambda row: parse_weights(row['Weight'], row['sets']), axis=1)
    
    # Calculate volume
    df['volume_per_set'] = df.apply(
        lambda row: [r * w for r, w in zip(row['reps_list'], row['weights_list'])],
        axis=1
    )
    df['total_volume'] = df['volume_per_set'].apply(sum)
    df['max_weight'] = df['weights_list'].apply(lambda x: max(x) if x else 0)
    
    return df

# Test data processing
processed_data = process_workout_data(sheets_data)
print("\nSample of processed data:")
processed_data


Sample of processed data:


,Date,Exercise,Reps,Weight,Time seconds,date,reps_list,weights_list,volume_per_set,total_volume,max_weight,week,year,sets
0,22/02/2025,Negative pull-up,7;7;6;7,Body;Body;Body;BodyBand,,2025-02-22,"[7, 7, 6, 7]","[120, 120, 120, 130]","[840, 840, 720, 910]",3310.0,130.0,8,2025,4
1,22/02/2025,Standing Cable Fly,8; 8; 8; 8; 8,17.5; 27.5; 27.5; 17.5; 12.5,,2025-02-22,"[8, 8, 8, 8, 8]","[17.5, 27.5, 27.5, 17.5, 12.5]","[140.0, 220.0, 220.0, 140.0, 100.0]",820.0,27.5,8,2025,5
2,22/02/2025,Seated Cable Row,10; 10; 10; 9,100; 100;100;100,,2025-02-22,"[10, 10, 10, 9]","[100.0, 100.0, 100.0, 100.0]","[1000.0, 1000.0, 1000.0, 900.0]",3900.0,100.0,8,2025,4
3,22/02/2025,Lat Pulldowns,10;10;10;10,100; 85;85;85,,2025-02-22,"[10, 10, 10, 10]","[100.0, 85.0, 85.0, 85.0]","[1000.0, 850.0, 850.0, 850.0]",3550.0,100.0,8,2025,4
4,22/02/2025,Incline Abs,15;15;15,Body,,2025-02-22,"[15, 15, 15]","[70, 70, 70]","[1050, 1050, 1050]",3150.0,70.0,8,2025,3
5,22/02/2025,Plank\t,,,60,2025-02-22,[],[],[],0.0,0.0,8,2025,0
6,23/02/2025,Leg Press Machine,10;10;13;20,180;270;270;270,,2025-02-23,"[10, 10, 13, 20]","[180.0, 270.0, 270.0, 270.0]","[1800.0, 2700.0, 3510.0, 5400.0]",13410.0,270.0,8,2025,4
7,23/02/2025,Leg Extension Machine,10;10;10;10,70;80;90;90,,2025-02-23,"[10, 10, 10, 10]","[70.0, 80.0, 90.0, 90.0]","[700.0, 800.0, 900.0, 900.0]",3300.0,90.0,8,2025,4
8,23/02/2025,Outer Thigh Machine,10;10;10;10,100;100;110;110,,2025-02-23,"[10, 10, 10, 10]","[100.0, 100.0, 110.0, 110.0]","[1000.0, 1000.0, 1100.0, 1100.0]",4200.0,110.0,8,2025,4
9,23/02/2025,Inner Thigh Machine,10;10;10;10,100;100;100;100,,2025-02-23,"[10, 10, 10, 10]","[100.0, 100.0, 100.0, 100.0]","[1000.0, 1000.0, 1000.0, 1000.0]",4000.0,100.0,8,2025,4


In [23]:
# processed_data.groupby('date')['Exercise'].count() # Avg Exercises/Session
# processed_data.groupby(['year', 'week']).size() # Sets per week

year  week
2025  8       11
      9       18
      10      17
      11      11
dtype: int64

## 3. Garmin Integration

Test the Garmin Connect integration and data processing.

In [12]:
# Initialize Garmin loader
garmin_loader = GarminDataLoader()

# Test connection
if garmin_loader.connect():
    print("Successfully connected to Garmin!")
    
    # Get last 7 days of data
    end_date = datetime.now()
    start_date = end_date - timedelta(days=7)
    
    garmin_data = garmin_loader.get_combined_workout_data(
        start_date.strftime("%Y-%m-%d"),
        end_date.strftime("%Y-%m-%d")
    )
    
    print(f"\nFound {len(garmin_data)} workouts from Garmin")
    print("\nSample of Garmin data:")
    garmin_data.head()
else:
    print("Failed to connect to Garmin. Check your credentials.")

Successfully connected to Garmin!

Found 0 workouts from Garmin

Sample of Garmin data:


## 4. Data Analysis Functions

Test the functions that calculate workout statistics and personal records.

In [13]:
def calculate_streak(df):
    """Calculate workout streak"""
    df['week'] = df['date'].dt.isocalendar().week
    df['year'] = df['date'].dt.isocalendar().year
    weekly_counts = df.groupby(['year', 'week']).size().reset_index(name='workout_count')
    
    streak = 0
    current_streak = 0
    for _, row in weekly_counts.sort_values(['year', 'week']).iterrows():
        if row['workout_count'] >= 5:
            current_streak += 1
            streak = max(streak, current_streak)
        else:
            current_streak = 0
    
    return streak, weekly_counts

def calculate_pr_table(df):
    """Calculate personal records table"""
    prs = []
    
    for exercise in df['Exercise'].unique():
        exercise_df = df[df['Exercise'] == exercise]
        
        for _, row in exercise_df.iterrows():
            for reps, weight in zip(row['reps_list'], row['weights_list']):
                prs.append({
                    'Exercise': exercise,
                    'Reps': reps,
                    'Weight': weight,
                    'Date': row['date']
                })
    
    pr_df = pd.DataFrame(prs)
    pr_df['Volume'] = pr_df['Reps'] * pr_df['Weight']
    
    max_weight_prs = pr_df.sort_values('Weight', ascending=False).groupby('Exercise').first()
    max_volume_prs = pr_df.sort_values('Volume', ascending=False).groupby('Exercise').first()
    
    return max_weight_prs, max_volume_prs

# Test streak calculation
streak, weekly_counts = calculate_streak(processed_data)
print(f"Current workout streak: {streak} weeks")

# Test PR calculation
max_weight_prs, max_volume_prs = calculate_pr_table(processed_data)
print("\nMax Weight PRs:")
print(max_weight_prs[['Weight', 'Reps', 'Date']])

print("\nMax Volume PRs:")
print(max_volume_prs[['Volume', 'Reps', 'Weight', 'Date']])

Current workout streak: 4 weeks

Max Weight PRs:
                                                Weight  Reps       Date
Exercise                                                               
Assisted Planche Dips                             70.0    10 2025-03-11
Back Extension Machine                            70.0    15 2025-03-06
Barbell squats                                   120.0    10 2025-03-06
Bench press                                      140.0    10 2025-03-08
Bench press incline                               80.0     5 2025-03-13
Bottom up Cable Crossover                          5.0     6 2025-02-25
Burpees                                           70.0    55 2025-03-11
Decline pushup                                    70.0    10 2025-02-27
Dips + Knee ups                                   70.0     5 2025-02-25
Dumbbell Pullover                                 45.0    10 2025-03-05
Dumbbell curls                                    30.0    10 2025-02-26
Dumbbell latera

## 5. Visualization Components

Test the visualization functions that create our interactive charts.

In [14]:
def create_streak_visualization(weekly_counts):
    """Create streak visualization"""
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=weekly_counts.apply(lambda x: f"{x['year']}-W{x['week']}", axis=1),
        y=weekly_counts['workout_count'],
        marker_color=weekly_counts['workout_count'].apply(
            lambda x: '#28a745' if x >= 5 else '#dc3545'
        ),
        name='Workouts'
    ))
    
    fig.update_layout(
        title='Weekly Workout Streak',
        xaxis_title='Week',
        yaxis_title='Workout Count',
        yaxis_range=[0, 7],
        height=300
    )
    
    return fig

def create_progress_chart(df, exercise_name):
    """Create progress chart for specific exercise"""
    if df.empty or exercise_name not in df['Exercise'].unique():
        return None
        
    exercise_data = df[df['Exercise'] == exercise_name].copy()
    
    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(
            x=exercise_data['date'],
            y=exercise_data['max_weight'],
            name='Max Weight',
            line=dict(color='#0068c9', width=2)
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=exercise_data['date'],
            y=exercise_data['total_volume'],
            name='Total Volume',
            yaxis='y2',
            line=dict(color='#28a745', width=2, dash='dot')
        )
    )
    
    fig.update_layout(
        title=f'{exercise_name} Progress',
        xaxis_title='Date',
        yaxis_title='Max Weight (kg)',
        yaxis2=dict(
            title='Total Volume',
            overlaying='y',
            side='right'
        ),
        height=400,
        showlegend=True,
        legend=dict(
            yanchor='top',
            y=0.99,
            xanchor='left',
            x=0.01
        ),
        hovermode='x unified'
    )
    
    return fig

# Test visualizations
streak_fig = create_streak_visualization(weekly_counts)
streak_fig.show()

# Test progress chart for a specific exercise
exercise_name = processed_data['Exercise'].iloc[0]  # Get first exercise as example
progress_fig = create_progress_chart(processed_data, exercise_name)
if progress_fig:
    progress_fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

## 6. Combining Data Sources

Test the function that combines data from Google Sheets and Garmin.

In [15]:
def combine_workout_data(sheets_df, garmin_df):
    """Combine data from Google Sheets and Garmin"""
    if garmin_df.empty:
        return sheets_df
        
    # Process Garmin data to match sheets format
    garmin_processed = garmin_df.copy()
    garmin_processed = garmin_processed.rename(columns={
        'exercise_name': 'Exercise',
        'weight': 'Weight',
        'reps': 'Reps'
    })
    
    # Convert weights and reps to lists format
    garmin_processed['weights_list'] = garmin_processed['Weight'].apply(lambda x: [float(x)] if pd.notnull(x) else [])
    garmin_processed['reps_list'] = garmin_processed['Reps'].apply(lambda x: [int(x)] if pd.notnull(x) else [])
    
    # Calculate volume
    garmin_processed['volume_per_set'] = garmin_processed.apply(
        lambda row: [r * w for r, w in zip(row['reps_list'], row['weights_list'])],
        axis=1
    )
    garmin_processed['total_volume'] = garmin_processed['volume_per_set'].apply(sum)
    garmin_processed['max_weight'] = garmin_processed['weights_list'].apply(lambda x: max(x) if x else 0)
    
    # Combine dataframes
    combined_df = pd.concat([sheets_df, garmin_processed[[col for col in sheets_df.columns if col in garmin_processed.columns]]], 
                          ignore_index=True)
    
    # Sort by date and remove duplicates
    combined_df = combined_df.sort_values('date').drop_duplicates(
        subset=['date', 'Exercise', 'Reps', 'Weight'], 
        keep='last'
    )
    
    return combined_df

# Test combining data sources if both are available
if not garmin_data.empty and not processed_data.empty:
    combined_data = combine_workout_data(processed_data, garmin_data)
    print(f"Combined data has {len(combined_data)} rows")
    print("\nSample of combined data:")
    combined_data[['date', 'Exercise', 'reps_list', 'weights_list', 'total_volume']].head()

Things to fix:
- Garmin data not integrated
- Need checks on Google sheets data (what if I fucked something up)
- Google sheets time column not implemented for time exercises
- improve main stats and charts to motivate me the most